In [9]:
# 교재는 tensorflow로 되어 있지만 우리는 파이토치를 사용.
# 복습떄 탠서플로우 사용해도 됨

### 🧠 RNN (Recurrent Neural Network) 요약정리

---

#### 📌 RNN 기본 구조

| 구분 | 설명 |
|------|------|
| **baseline** | 일반 신경망 (Dense Layer) |
| **Simple RNN** | 기울기 소실(Vanishing Gradient) 문제 존재 |
| **LSTM** | RNN의 기울기 소실 문제 해결 |
| **Bidirectional LSTM** | 양방향으로 문맥 학습 |

> 🔹 **RNN**은 순서(시퀀스)가 있는 데이터를 다룰 때 사용한다.  
> 예: 문장, 음성, 시계열 데이터

---

#### 📚 텍스트 전처리 과정

| 단계 | 설명 | 관련 함수 |
|------|------|-----------|
| **1. Tokenization** | 텍스트를 숫자로 변환 (단어별 인덱스 생성) | `Tokenizer()` |
| **2. Word Embedding** | 단어를 고차원 실수 벡터로 변환 | `Embedding()` |
| **3. Sequence Padding** | 문장 길이를 동일하게 맞춤 | `pad_sequences()` |

---

#### ⚙️ 토크나이저 3단계 요약

| 단계 | 함수 | 기능 |
|------|------|------|
| ① | `fit_on_texts(texts)` | 가장 많이 등장한 단어 순서대로 **인덱스 딕셔너리 구축** |
| ② | `texts_to_sequences(texts)` | 각 문서를 **정수 시퀀스**로 변환 |
| ③ | `pad_sequences(sequences)` | 길이를 맞추어 **패딩 처리 (동일한 길이)** |

> 💡 `Tokenizer`는 단어 → 숫자 인코딩,  
> `Embedding`은 숫자 → 벡터 인코딩 역할을 함.

---

#### 🧩 임베딩(Embedding) Layer

| 항목 | 설명 |
|------|------|
| **정의** | 단어를 고정된 크기의 실수 벡터로 변환하는 층 |
| **입력값** | 정수 인덱스 시퀀스 (토큰화 결과물) |
| **출력값** | 각 단어의 벡터 표현 (예: 128차원, 300차원 등) |
| **역할** | 모델이 단어 간 의미적 유사도를 학습할 수 있도록 함 |

> 예시:  
> `"dog"` → `[0.12, 0.45, -0.33, ...]`  
> `"cat"` → `[0.10, 0.43, -0.31, ...]`  
> (비슷한 의미의 단어는 벡터 공간상에서도 가깝게 위치)

---

#### 🧱 모델 종류별 비교

| 모델 | 특징 | 장단점 |
|------|------|--------|
| **Simple RNN** | 기본 RNN 구조 | 계산 단순하지만 장기 의존성 학습 어려움 |
| **LSTM** | 셀 상태(cell state)로 장기 기억 유지 | 복잡하지만 성능 우수 |
| **Bidirectional LSTM** | 문장을 앞뒤 양방향으로 학습 | 문맥 파악 정확도 높음, 계산량 많음 |

---

#### 🔁 RNN 학습의 흐름

```mermaid
graph TD
A[텍스트 데이터] --> B[토큰화 (Tokenizer)]
B --> C[정수 시퀀스 변환]
C --> D[패딩 (pad_sequences)]
D --> E[임베딩 레이어]
E --> F[Simple RNN / LSTM / Bi-LSTM]
F --> G[Dense Layer 출력]


In [10]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from collections import Counter
import numpy as np
import torch


sample_reviews = [
    "this movie is great and wonderful",
    "bad movie with poor acting",
    "great movie absolutely wonderful"
]

# 1. 단어 분할 및 빈도 계산
all_reviews = []
# [ all_reviews.extend(i) for i in [review.split() for review in sample_reviews] ]

for i in [review.split() for review in sample_reviews]:
    all_reviews.extend(i)
    
# 2. 단어빈도
word_freq = Counter(all_reviews)

# 3. Tokenizer 구현
class SimpleTokenizer:
    def __init__(self , num_reviews=10 , oov_token='UNK'):
        self.num_reviews = num_reviews
        self.oov_token = oov_token
        self.review_index = {}
        self.index_review = {}
    def fit_on_texts(self , texts):
        all_reviews = []
        for i in [review.split() for review in sample_reviews]:
            all_reviews.extend(i)
        word_freq = Counter(all_reviews)
        # 빈도수가 높은 순서로 index 부여
        # oov 토큰을 1로 설정
        self.review_index[self.oov_token] = 1
        self.index_review[1] = self.oov_token
        idx = 2
        for word , _ in word_freq.most_common(self.num_reviews-1):
            self.review_index[word] = idx
            self.index_review[idx] = word
            idx += 1
    def texts_to_sequences(self , texts):
        '''텍스트를 점수 시퀀스로 변환'''
        sequentces = []
        for text in texts:
            seq = []
            for word in text.split():
                # 단어가 vocabulary 에 있으면 index를 사용 / 없으면 oov
                review_index = self.review_index.get(word,1)
                seq.append(review_index)
            sequentces.append(seq)
        return sequentces
    
# Tokenizer  생성 및 학습
tokenizer = SimpleTokenizer(num_reviews=10 , oov_token='UNK')
tokenizer.fit_on_texts(sample_reviews)
tokenizer.review_index

# 텍스트를 시퀀스로 변환
tokenizer.texts_to_sequences(sample_reviews) # 3개의 문장을 시퀀스로 변환함 ( 문자의 길이가 다 달라)

[[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]

In [14]:
# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(sample_reviews)
sequences

[[5, 2, 6, 3, 7, 4], [8, 2, 9, 10, 1], [3, 2, 1, 4]]

In [15]:
sample_reviews

['this movie is great and wonderful',
 'bad movie with poor acting',
 'great movie absolutely wonderful']

In [16]:
# 패딩 구현 - 문자열의 길이를 동일하게 맞춘다
def pad_sequence_manual(sequences, max_len=10, padding='pre',value=0):
    '''패딩구현'''
    padded = []
    for seq in sequences:
        if len(seq) >= max_len:
            if padded == 'pre':
                padded_seq = seq[-max_len:]
            else:
                padded_seq = seq[:max_len]
        else:
            pad_length = max_len-len(seq)
            if padding == 'pre':
                padded_seq = [value]*pad_length + seq
            else:
                padded_seq = seq + [value]*pad_length
        padded.append(padded_seq)
    return np.array(padded)
padded = pad_sequence_manual(sequences)
padded

array([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
       [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
       [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [17]:
# pytorch tensor 변환
sequence_tensor =  torch.LongTensor(padded)
sequence_tensor

tensor([[ 0,  0,  0,  0,  5,  2,  6,  3,  7,  4],
        [ 0,  0,  0,  0,  0,  8,  2,  9, 10,  1],
        [ 0,  0,  0,  0,  0,  0,  3,  2,  1,  4]])

In [ ]:
# 2. 워드 임베딩 - 시각화
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.nn as nn
print(f'패딩된 시쿼스 형태 : {sequence_tensor.shape}')
print(f'첫번째 : {sequence_tensor[0]}')
# pytorch embedding 레이어 생성
# num_embeddings 어휘의 크기
# embedding_dim 각 단어를 몇차원 벡터로 표현할 것인지
# padding_idx 길이맞출때 채우는 값
embedding_layer = nn.Embedding(num_embeddings=1000,embedding_dim=8,padding_idx=0)
embedded = embedding_layer(sequence_tensor)
print(f'입력형태 : {sequence_tensor.shape}')
print(f'출력형태 : {embedded.shape}')

패딩된 시쿼스 형태 : torch.Size([3, 10])
첫번째 : tensor([0, 0, 0, 0, 5, 2, 6, 3, 7, 4])


In [ ]:
# 임베팅 벡터 상세 분석
# 샘플 데이터의 첫 3개 단어 임베딩
for word_idx in range(3):
    embedding_vec =  embedded[0,word_idx].detach().numpy()
    word_id = sequence_tensor[0, word_idx].item()
    print(f'단어 id { word_id} : {embedding_vec} -- 8차원중 처음 4개')

In [ ]:
# 임베딩 행렬
embedding_matrix =  embedding_layer.weight.detach().numpy()
print(f'임베딩 행렬 형태 : {embedding_matrix.shape}')  # 1000,8
print(f'패딩(id=0)의 임베딩 {embedding_matrix[0]}')
print(f'단어 id=5인 임베딩 {embedding_matrix[5]}')